# downsample microscopy and save locally


In [ ]:
import pandas as pd
import os
from os.path import split,join,splitext
import tifffile
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import re


### Setup list of file names

In [ ]:
# data location root directory
root_dir = '/home/brysongray/bmap/dong/hanpeng-bk-SF221013_02A/'
spreadsheet = 'SF221013_02A_converted_register list.xlsx'
myelin_dir = 'SF221013_02A_BK_converted'
fluoro_dir = 'SF221013_02A_converted'
n_fluoro_channels = 5
# location we would like to save the registered images
outdir = '/home/brysongray/data/hongwei_data/SF221013_02A/'


In [ ]:
# get lists of file names from the excel sheet
fnames_data = pd.read_excel(os.path.join(root_dir, spreadsheet), usecols=[0,2])
filesM = fnames_data.iloc[:,0] # Myelin
filesN = fnames_data.iloc[:,1] # Fluoro
# remove nan values from the dataframe
filesM = filesM.dropna()
filesN = filesN.dropna()
# remove single space strings from the dataframe
filesM = filesM[filesM != ' ']
filesN = filesN[filesN != ' ']
# replace double backslash with single forward slash
filesM = [f.replace('\\', '/') for f in filesM]
filesN = [f.replace('\\', '/') for f in filesN]
# get only the file names
filesM = [split(f)[1][1:] for f in filesM] # remove the first character which is a space
filesN = [split(f)[1][2:] for f in filesN] # remove the first two characters which are spaces

# compare the file names in the excel sheet to the file names in the directory
listdirN = os.listdir(join(root_dir, fluoro_dir))
listdirN = [f for f in listdirN if 'Overview' not in f]
listdirM = os.listdir(join(root_dir, myelin_dir))
listdirM = [f for f in listdirM if 'Overview' not in f]
# check that listdirN and filesN contain the same files
filesNintersection = [f for f in filesN if f in listdirN]
print(len(filesNintersection), len(listdirN), len(filesN))
print(len(filesNintersection)==len(listdirN)==len(filesN))
# check that listdirM and filesM contain the same files
filesMintersection = [f for f in filesM if f in listdirM]
print(len(filesMintersection), len(listdirM), len(filesM))
print(len(filesMintersection)==len(listdirM)==len(filesM))

# filesN must be grouped into n for each image with n channels
filesN0 = [filesN[i:i+n_fluoro_channels] for i in range(0, len(filesN[:74]), n_fluoro_channels)]
filesN1 = [filesN[i:i+n_fluoro_channels-1] for i in range(75, len(filesN), n_fluoro_channels-1)]
filesN = filesN0 + filesN1

### display sample images

In [ ]:
I = []
for f in filesN[0]:
    with tifffile.TiffFile(os.path.join(root_dir, fluoro_dir, f)) as tif:
        img = tif.asarray()[::2,::2]
        img = img.astype(float)/img.max().astype(float)
        I.append(img)
        
with tifffile.TiffFile(os.path.join(root_dir, myelin_dir, filesM[0])) as tif:
    J = tif.asarray()[::4,::4]
    J = J.astype(float)/J.max().astype(float)



In [ ]:
print(J.dtype)
print(J.max())
print(I[0].dtype)
print(I[0].max())

In [ ]:
i=0
print(I[i].shape)
print(J.shape)

fig, axs = plt.subplots(2,2)
axs[0,0].imshow(I[0])
axs[0,0].set_title('Cy5')
axs[0,1].imshow(I[1])
axs[0,1].set_title('GFP')
axs[1,0].imshow(I[2])
axs[1,0].set_title('DAPI')
axs[1,1].imshow(I[3])
axs[1,1].set_title('GOLD')
    


In [ ]:
I_rgb = np.stack(I[:3],axis=-1)
fig = plt.figure()
plt.imshow(I_rgb)
plt.title('Cy5-GFP-DAPI as RGB')

In [ ]:
plt.figure()
plt.imshow(J)
plt.title('Myelin')

### Loop through files, downsample and save

### keep the first three channels of fluoro

In [ ]:
filesN[0][0].split('CH')[0]+'down2.tif'

In [ ]:
for fileN in filesN:
    I = []
    for f in fileN[:3]:
        with tifffile.TiffFile(os.path.join(root_dir, fluoro_dir, f)) as tif:
            dx = tif.pages[0].tags.values()[19].value['pixelsizex']
            dy = tif.pages[0].tags.values()[19].value['pixelsizey']
            I.append(tif.asarray()[::2,::2])
    I = np.stack(I, -1)
    fname = fileN[0].split('CH')[0]+f'down2.tif'
    print(f'saving: {os.path.join(outdir, "fluoro", fname)}')
    tifffile.imwrite(os.path.join(outdir, 'fluoro', fname), I, metadata={'pixelsizex':dx*2, 'pixelsizey':dy*2})

In [ ]:
for fileM in filesM:
    with tifffile.TiffFile(os.path.join(root_dir, myelin_dir, fileM)) as tif:
        dx = tif.pages[0].tags.values()[19].value['pixelsizex']
        dy = tif.pages[0].tags.values()[19].value['pixelsizey']
        I = tif.asarray()[::4,::4]
    fname = fileM.split('.tif')[0]+'_down4.tif'
    print(f'saving: {os.path.join(outdir, "bk", fname)}')
    tifffile.imwrite(os.path.join(outdir, 'bk', fname), I, metadata={'pixelsizex':dx*4, 'pixelsizey':dy*4})

In [ ]:
# correct the numbering so the images align when the lists are sorted
# filesM = sorted(os.listdir(join(outdir, 'bk')))
filesM = sorted(os.listdir(join(root_dir, myelin_dir)))
filesM = [f for f in filesM if 'Overview' not in f]
print(filesM[:15])
filesM_v1 = [] 
for f in filesM:
    fsplit = re.split(r'(D_.._)',f)
    if fsplit[2][:3] == '20x':
        f_ = fsplit[0] + fsplit[1] + '01_' + fsplit[2]
        filesM_v1.append(f_)
    else:
        # n = int(f.split('_AD_0*_')[1][:2])
        n = int(fsplit[2][:2])
        f_ = fsplit[0] + fsplit[1] + f'{n+1:02}' + fsplit[2][2:]
        filesM_v1.append(f_)
print(filesM_v1[:15])


In [ ]:
root = join(root_dir, myelin_dir)
for f in range(len(filesM)):
    os.rename(join(root,filesM[f]), join(root,filesM_v1[f]))